## Challenge 1

Calculates the number of water molecules within 4 Angstroms of the protein during each timestep.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    graph = LiveTimeseriesGraph(
        time_window=1.0, # ps
        dt=0.010, # ps
        title='Number of water molecules within 4 Angstroms of protein vs. Time',
        y_label='Number of water molecules',
    )

    for ts in u.trajectory:
        # Select all water molecules that are within 4 angstroms of a non-water (protein) molecule
        sel = u.select_atoms("(resname SOL) and (around 4 not resname SOL)")
        graph.update(ts.time, len(sel))

except Exception as e:
    print(e)
finally:
    u.trajectory.close()

## Challenge 2

Calculate the radius of gyration of the protein for each timestep.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
import numpy as np
from graph_utils import LiveTimeseriesGraph

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:9999", buffer_size=100*1024**2)

try:

    graph = LiveTimeseriesGraph(
        time_window=1.0, # ps
        dt=0.010, # ps
        title='Radius of gyration vs. Time',
        y_label='Radius of gyration (Angstroms)',
    )

    sel = u.select_atoms("protein")
    total_mass = np.sum(u.atoms.masses)

    for ts in u.trajectory:
        
        center_of_mass = sel.center_of_mass()
        pos = sel.positions
        masses = sel.masses

        ri_sq = np.square((pos - center_of_mass))
        ri_sq = np.sum(ri_sq, axis=1)
        sq = np.sum(ri_sq * masses)
        sq = sq / total_mass

        radius_of_gyration = np.sqrt(sq)

        graph.update(ts.time, radius_of_gyration)

except Exception as e:
    print(e)
finally:
    u.trajectory.close()